# SKEMA-TA1-UAZ-MORAE Demo (2022-12-08)

**Authors**: Justin Lieffers, Liang Zhang, Deepsana Shahi, Adarsh Pyarelal, Clayton Morrison

In [ ]:
# Set the base URL
BASE_URL="http://localhost:8080"

In [ ]:
from IPython.display import IFrame
IFrame(f"{BASE_URL}/docs/", width=700, height=350)

## Model role identification

One of the prerequisites for model role identification is the ability to efficiently query GroMEts, including path traversal queries. For this reason, we have adopted a graph database backend for storing the structure inherent in GroMEt JSON files, along with additional metadata to speed up queries.

Below, we demonstrate how to interact with this database using our REST API.

First, we perform our imports and set the base URL

In [ ]:
import json
from requests import get, put, post, delete

Then, we check if the service is running.

In [ ]:
get(f"{BASE_URL}/ping").text

We now have functionality to ingest GroMEt JSON files (still working on adding domain coverage for this pipeline). Below we take `sir_core--Gromet-FN-auto.json` and return AMR JSONs. This gromet comes from a core dynamics python script of an SIR model. There exists endpoints for PetriNets (PN) and RegNets (RN). However, RN's aren't fully supported yet as more test cases are needed. 

These are currently "barebones" AMR's in that it just has the structural data, but are something to build and expand on. 

In [ ]:
with open('data/sir_core--Gromet-FN-auto.json') as f:
    r = put(f"{BASE_URL}/models/PN", json=json.load(f))
r.json()

There is also support for converting mathml into AMR's, either PetriNets or RegNets again. Once again only "barebones" AMR's immediately. 

Below is for regnets using lotka voltera mathml

In [ ]:
mathml_vec = []

with open("data/lotka_voltera/mml_list.txt") as file:
    for line in file:
        mathml_vec.append(line.rstrip())

print(json.dumps(mathml_vec))

headers = {
    "Content-Type": "application/json"
}

r = put(f"{BASE_URL}/mathml/regnet", data=json.dumps(mathml_vec), headers=headers)
r.json()

We can also generate for a petrinet, below is an example with sir mathml

In [ ]:
mathml_vec = []

with open("data/simple_sir_v1/mml_list.txt") as file:
    for line in file:
        mathml_vec.append(line.rstrip())

print(json.dumps(mathml_vec))

headers = {
    "Content-Type": "application/json"
}

r = put(f"{BASE_URL}/mathml/petrinet", data=json.dumps(mathml_vec), headers=headers)
r.json()

Lastly we also have support to route the mathml based on the model type if given as json object as the following:

In [ ]:
class MathmlModel:
    def __init__(self, mathml, model):
        self.mathml = mathml
        self.model = model
    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__, 
            sort_keys=True, indent=4)

mathml_vec = []

with open("data/lotka_voltera/mml_list.txt") as file:
    for line in file:
        mathml_vec.append(line.rstrip())

lotka_voltera = MathmlModel(mathml_vec, "regnet")

headers = {
    "Content-Type": "application/json"
}

print(lotka_voltera.toJSON())

r = put(f"{BASE_URL}/mathml/amr", data=lotka_voltera.toJSON(), headers=headers)
r.json()